In [1]:
# Install dependencies (Uncomment if running in Colab)
# !pip install pandas scikit-learn joblib

# === Import libraries ===
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error
import joblib

# === Load dataset ===
df = pd.read_csv("/content/Dataset.csv.csv")

# === Rename columns ===
df.columns = [
    "Timestamp", "Name", "ScreenTime", "Meals", "Bedtime",
    "WakeupTime", "PhysicalActivity", "Stress"
]

# === Drop unnecessary columns ===
df = df.drop(columns=["Timestamp", "Name"])

# === Remove non-numeric entries ===
df = df[~df["ScreenTime"].str.lower().str.contains("phone", na=False)]
df["ScreenTime"] = pd.to_numeric(df["ScreenTime"], errors='coerce')
df["Meals"] = pd.to_numeric(df["Meals"], errors='coerce')

# === Convert HH:MM to float ===
def time_to_float(t):
    try:
        h, m = map(int, t.split(":"))
        return h + m / 60
    except:
        return np.nan

df["Bedtime"] = df["Bedtime"].apply(time_to_float)
df["WakeupTime"] = df["WakeupTime"].apply(time_to_float)

# === Map Physical Activity ===
activity_map = {"Low": 1, "Moderate": 3, "High": 5}
df["PhysicalActivity"] = df["PhysicalActivity"].map(activity_map)

# === Convert Stress to numeric ===
df["Stress"] = pd.to_numeric(df["Stress"], errors='coerce')

# === Simulate Sleep Quality Score (Target) ===
np.random.seed(42)
df["SleepQuality"] = (
    10
    - 0.5 * df["ScreenTime"]
    + 0.4 * df["Meals"]
    - 0.3 * (24 - (df["WakeupTime"] - df["Bedtime"]).abs())
    + 0.5 * df["PhysicalActivity"]
    - 0.4 * df["Stress"]
    + np.random.normal(0, 1, len(df))
).clip(1, 10).round(2)

# === Drop missing values ===
df = df.dropna()

# === Prepare data ===
X = df.drop("SleepQuality", axis=1)
y = df["SleepQuality"]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# === Train model ===
model = LinearRegression()
model.fit(X_train, y_train)

# === Evaluate model ===
y_pred = model.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)
mae = mean_absolute_error(y_test, y_pred)

print(f"\n📊 Evaluation Metrics:")
print(f"RMSE: {rmse:.2f}")
print(f"MAE: {mae:.2f}")

# === Save model ===
joblib.dump(model, "model.pkl")
print("✅ Model saved as 'model.pkl'")

# === Interactive Prediction ===
print("\n📝 Enter your lifestyle details to predict sleep quality:\n")

try:
    screen_time = float(input("Enter daily screen time (hours): "))
    meals = int(input("Enter number of meals per day: "))
    bedtime = float(input("Enter bedtime (24-hour format, e.g., 22.5 for 10:30 PM): "))
    wake_time = float(input("Enter wake-up time (24-hour format, e.g., 7.0 for 7 AM): "))
    activity = int(input("Enter physical activity level (1=Low, 3=Moderate, 5=High): "))
    stress = int(input("Enter stress level (1 to 5): "))

    # Predict sleep quality
    user_sample = [[screen_time, meals, bedtime, wake_time, activity, stress]]
    predicted_score = model.predict(user_sample)[0]

    print(f"\n✅ Predicted Sleep Quality Score: {round(predicted_score, 2)} / 10")
except Exception as e:
    print("❌ Error:", e)


📊 Evaluation Metrics:
RMSE: 1.04
MAE: 0.86
✅ Model saved as 'model.pkl'

📝 Enter your lifestyle details to predict sleep quality:

Enter daily screen time (hours): 5
Enter number of meals per day: 3
Enter bedtime (24-hour format, e.g., 22.5 for 10:30 PM): 22.25
Enter wake-up time (24-hour format, e.g., 7.0 for 7 AM): 6.0
Enter physical activity level (1=Low, 3=Moderate, 5=High): 3
Enter stress level (1 to 5): 3

✅ Predicted Sleep Quality Score: 6.26 / 10


/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LinearRegression was fitted with feature names
  warnings.warn(
